<a href="https://colab.research.google.com/github/PauloMarvin/ETL-CVM/blob/data-analysis/03_refined_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando dependências do *Spark*

In [2]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

Importando OS

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

Importando *SparkSession*

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [6]:
spark

Importando bibliotecas

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px
import seaborn as sns

In [8]:
sns.set()

# Carregando os dados no *dataframe*

In [9]:
df = spark.read.csv(
    '/content/drive/MyDrive/cvm_2000_2022.csv',
    header= True, inferSchema= True
)

In [10]:
df.show(5)

+---+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+
|_c0|TP_FUNDO|        CNPJ_FUNDO| DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|
+---+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+
|  0|   FITVM|01.465.738/0001-97|2000-01-03|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|
|  1|   FITVM|01.465.738/0001-97|2000-01-04|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|
|  2|   FITVM|01.465.738/0001-97|2000-01-05|    null|2.4035701|    2467000.0|  44000.0|  2000.0|    null|
|  3|   FITVM|01.465.738/0001-97|2000-01-06|    null|2.4648947|    2573000.0|  21000.0|     0.0|    null|
|  4|   FITVM|01.465.738/0001-97|2000-01-07|    null|2.5131104|    2646000.0|   3000.0|     0.0|    null|
+---+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+
only showing top 5 rows



## Renomeação da coluna `_c0`

In [11]:
df = df.withColumnRenamed('_c0', 'index')

In [12]:
df.show(5)

+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+
|index|TP_FUNDO|        CNPJ_FUNDO| DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+
|    0|   FITVM|01.465.738/0001-97|2000-01-03|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|
|    1|   FITVM|01.465.738/0001-97|2000-01-04|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|
|    2|   FITVM|01.465.738/0001-97|2000-01-05|    null|2.4035701|    2467000.0|  44000.0|  2000.0|    null|
|    3|   FITVM|01.465.738/0001-97|2000-01-06|    null|2.4648947|    2573000.0|  21000.0|     0.0|    null|
|    4|   FITVM|01.465.738/0001-97|2000-01-07|    null|2.5131104|    2646000.0|   3000.0|     0.0|    null|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+
only showing top 5 rows



## Dimensões do *dataframe*

In [13]:
print(f'Há no dataframe {len(df.columns)} colunas.')
print(f'E {df.select("TP_FUNDO").count()} linhas.')

Há no dataframe 10 colunas.
E 57676391 linhas.


## *Schema* do *dataframe*

In [14]:
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- TP_FUNDO: string (nullable = true)
 |-- CNPJ_FUNDO: string (nullable = true)
 |-- DT_COMPTC: string (nullable = true)
 |-- VL_TOTAL: double (nullable = true)
 |-- VL_QUOTA: double (nullable = true)
 |-- VL_PATRIM_LIQ: double (nullable = true)
 |-- CAPTC_DIA: double (nullable = true)
 |-- RESG_DIA: double (nullable = true)
 |-- NR_COTST: double (nullable = true)



# Derivação de novas colunas a partir da coluna `DT_COMPTC`

### Colunas `MONTH` e `YEAR`

In [15]:
import pyspark.sql.functions as f

In [16]:
df = df.withColumn('MONTH', f.month('DT_COMPTC'))\
       .withColumn('YEAR', f.year('DT_COMPTC'))

In [17]:
df.show(2)

+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+
|index|TP_FUNDO|        CNPJ_FUNDO| DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+
|    0|   FITVM|01.465.738/0001-97|2000-01-03|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|    1|2000|
|    1|   FITVM|01.465.738/0001-97|2000-01-04|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|    1|2000|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+
only showing top 2 rows



### Coluna `MONTH_OF_YEAR`

In [18]:
df = df.withColumn('MONTH_OF_YEAR', f.date_format(df['DT_COMPTC'], 'yyyy-MM'))

In [19]:
df.show(2)

+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+
|index|TP_FUNDO|        CNPJ_FUNDO| DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+
|    0|   FITVM|01.465.738/0001-97|2000-01-03|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|    1|2000|      2000-01|
|    1|   FITVM|01.465.738/0001-97|2000-01-04|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|    1|2000|      2000-01|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+
only showing top 2 rows



### Coluna `QUARTER_OF_YEAR`

In [67]:
df = df.withColumn('QUARTER', f.concat(f.year('DT_COMPTC'), f.quarter('DT_COMPTC')))

In [68]:
df.show(2)

+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+-------+
|index|TP_FUNDO|        CNPJ_FUNDO| DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+-------+
|    0|   FITVM|01.465.738/0001-97|2000-01-03|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|    1|2000|      2000-01|  20001|
|    1|   FITVM|01.465.738/0001-97|2000-01-04|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|    1|2000|      2000-01|  20001|
+-----+--------+------------------+----------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+-------+
only showing top 2 rows



# Análise exploratória

## Quantidade de dados faltantes

In [25]:
df.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in df.columns]).show()

+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+
|index|TP_FUNDO|CNPJ_FUNDO|DT_COMPTC|VL_TOTAL|VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+
|    0|40860852|         0|        0|  366012|       0|            0|        0|       0|  365408|    0|   0|            0|      0|
+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+



In [26]:
df.agg(*[f.mean(f.when(f.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).show()

+-----+------------------+----------+---------+-------------------+--------+-------------+---------+--------+--------------------+-----+----+-------------+-------+
|index|          TP_FUNDO|CNPJ_FUNDO|DT_COMPTC|           VL_TOTAL|VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|            NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+-----+------------------+----------+---------+-------------------+--------+-------------+---------+--------+--------------------+-----+----+-------------+-------+
|  0.0|0.7084502218594086|       0.0|      0.0|0.00634595878233782|     0.0|          0.0|      0.0|     0.0|0.006335486559830...|  0.0| 0.0|          0.0|    0.0|
+-----+------------------+----------+---------+-------------------+--------+-------------+---------+--------+--------------------+-----+----+-------------+-------+



Há uma quantidade bastante expressiva de dados faltantes na coluna `TP_FUNDO`. Os dados faltantes respondem por, aproximadamente, 71% dos dados da coluna.

## Coluna `TP_FUNDO`

In [27]:
count_tp_fundo = df.groupBy('TP_FUNDO').count().toPandas().sort_values('count', ascending= False)

In [28]:
count_tp_fundo

,TP_FUNDO,count
2,None,40860852
0,FI,15266406
5,FIF,500219
10,FACFIF,492624
9,FITVM,349153
4,FMP-FGTS,107997
7,FIC-FITVM,67022
8,FIEX,13624
1,FAPI,9395
11,FMP-FGTS CL,8226


## Coluna `VL_TOTAL`

### Estatísticas descritivas

In [29]:
df.describe('VL_TOTAL').show(truncate= False)

+-------+---------------------+
|summary|VL_TOTAL             |
+-------+---------------------+
|count  |57310379             |
|mean   |3.530323818966721E8  |
|stddev |2.9902593485264565E10|
|min    |-5.24478822941659E12 |
|max    |2.15700309082785E14  |
+-------+---------------------+



In [30]:
avg_vl_total_per_date = df.groupBy('DT_COMPTC').agg(f.avg('VL_TOTAL')).sort('DT_COMPTC').toPandas()

Para identificar a existências de valores discrepantes plota-se um gráfico para observar as médias do `VL_TOTAL` em cada data:

In [31]:
fig = px.line(
    data_frame= avg_vl_total_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média diária da variável VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, xaxis_title= 'Dia', yaxis_title= 'Média do VL_TOTAL')
fig.show()

### *Boxplot* da variável `VL_TOTAL`

In [32]:
fig = px.box(avg_vl_total_per_date['avg(VL_TOTAL)'], title= 'Boxplot da variável VL_TOTAL', orientation= 'h')
fig.update_layout(title= {'x': 0.5}, yaxis_title= '', xaxis_title= 'Valor')
fig.show()

### Remoção de *outliers* da variável

In [33]:
Q1 = df.approxQuantile('VL_TOTAL', [0.25], 0.01)[0] # os parâmetros para o método approxQuantile são, respectivamente: coluna, quantil e erro.
Q3 = df.approxQuantile('VL_TOTAL', [0.75], 0.01)[0] # é retornada uma lista com o valor, por isso utiliza-se o [0], para obter apenas o valor.
IIQ = Q3 - Q1

In [34]:
inferior = Q1 - (1.5 * IIQ)
superior = Q3 + (1.5 * IIQ)

In [69]:
vl_total_without_outliers = df.where((df['VL_TOTAL'] >= inferior) & (df['VL_TOTAL'] <= superior) & (df['VL_TOTAL'] != 0)) # removem-se os dados onde a média
# é zero.

## Análises após a remoção dos *outliers*

### Estatísticas descritivas

In [36]:
vl_total_without_outliers.describe('VL_TOTAL').show()

+-------+-------------------+
|summary|           VL_TOTAL|
+-------+-------------------+
|  count|           48691791|
|   mean|5.585767112200188E7|
| stddev|6.742780835377653E7|
|    min|    -1.6804662665E8|
|    max|     3.1408188281E8|
+-------+-------------------+



### Medidas de tendência central para variável `VL_TOTAL`

#### Média diária

In [37]:
avg_vl_total_per_date = vl_total_without_outliers.groupBy('DT_COMPTC').agg(f.avg('VL_TOTAL')).sort('DT_COMPTC').toPandas()

In [38]:
fig = px.line(
    data_frame= avg_vl_total_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média diária do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Dia')
fig.show()

#### Média mensal

In [39]:
avg_vl_total_per_month = vl_total_without_outliers.groupby('MONTH_OF_YEAR').agg(f.avg('VL_TOTAL')).orderBy('MONTH_OF_YEAR').toPandas()

In [40]:
fig = px.line(
    data_frame= avg_vl_total_per_month,
    x= 'MONTH_OF_YEAR',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média mensal do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Mês')
fig.show()

#### Média trimestral

In [70]:
avg_vl_total_per_quarter = vl_total_without_outliers.groupBy('QUARTER').agg(f.avg('VL_TOTAL')).orderBy('QUARTER').toPandas()

In [72]:
fig = px.line(
    data_frame= avg_vl_total_per_quarter,
    x= 'QUARTER',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média trimestral do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Trimestre')
fig.show()

#### Média anual

In [41]:
avg_vl_total_per_year = vl_total_without_outliers.groupBy('YEAR').agg(f.avg('VL_TOTAL')).orderBy('YEAR').toPandas()

In [42]:
fig = px.line(
    data_frame= avg_vl_total_per_year,
    x= 'YEAR',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média anual do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Ano')
fig.show()

## Coluna `VL_QUOTA`

### Estatísticas descritivas

In [43]:
df.describe('VL_QUOTA').show(truncate= False)

+-------+----------------------+
|summary|VL_QUOTA              |
+-------+----------------------+
|count  |57676391              |
|mean   |203815.64916116075    |
|stddev |1.4761400027915974E9  |
|min    |-1.2335447365092747E11|
|max    |1.1206620286175E13    |
+-------+----------------------+



In [73]:
avg_vl_quota_per_date = df.groupBy('DT_COMPTC').agg(f.avg('VL_QUOTA')).orderBy('DT_COMPTC').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_total_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média diária da variável VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, xaxis_title= 'Dia', yaxis_title= 'Média do VL_TOTAL')
fig.show()